エージェントの核心的なコンセプトは、一連のアクションを選択するために言語モデルを使用することです。チェーンでは、アクションのシーケンスはコード内でハードコードされています。エージェントでは、どのアクションを、どの順序で取るべきかを決定するために、言語モデルが推論エンジンとして使用されます。

> The core idea of agents is to use a language model to choose a sequence of actions to take.
> In chains, a sequence of actions is hardcoded (in code).
> In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

## Concepts | 概念

ここにはいくつかの重要な要素があります：

> There are several key components here:

### Agent | エージェント

これは次にどのステップを踏むかを決定する責任を持つチェーンです。これは言語モデルとプロンプトによって駆動されています。このチェーンへの入力は以下の通りです：

> This is the chain responsible for deciding what step to take next.
> This is powered by a language model and a prompt.
> The inputs to this chain are:

1. ツール: 利用可能なツールの説明

   > Tools: Descriptions of available tools

2. ユーザー入力：高水準の目的

   > User input: The high level objective

3. 中間ステップ：ユーザーの入力を達成するために以前に実行された（アクション、ツールの出力）のペア

   > Intermediate steps: Any (action, tool output) pairs previously executed in order to achieve the user input


出力は、次に取るべきアクション、またはユーザーに送信する最終的なレスポンスです（`AgentAction`または`AgentFinish`）。アクションは、ツールとそのツールに対する入力を指定します。

> The output is the next action(s) to take or the final response to send to the user (`AgentAction`s or `AgentFinish`). An action specifies a tool and the input to that tool.

異なるエージェントには、推論のための異なるプロンプトスタイル、入力のエンコーディング方法、出力のパース方法があります。組み込みエージェントの完全なリストについては、[エージェントタイプ](/docs/modules/agents/agent_types/)をご覧ください。また、カスタムエージェントを**簡単に構築することが可能です**。その方法については、下記の「はじめに」セクションで示しています。

> Different agents have different prompting styles for reasoning, different ways of encoding inputs, and different ways of parsing the output.
> For a full list of built-in agents see [agent types](/docs/modules/agents/agent_types/).
> You can also **easily build custom agents**, which we show how to do in the Get started section below.

### Tools | ツール

ツールはエージェントが呼び出すことができる機能です。
ツールに関する2つの重要な設計上の考慮事項があります：

> Tools are functions that an agent can invoke.
> There are two important design considerations around tools:

1. エージェントに適切なツールへのアクセスを与える

   > Giving the agent access to the right tools

2. エージェントにとって最も役立つ方法でツールを説明する

   > Describing the tools in a way that is most helpful to the agent


両方をしっかりと考え抜かなければ、機能するエージェントを構築することはできません。エージェントに正しいツールセットへのアクセスを与えなければ、あなたが設定した目標を達成することは決してできません。ツールを適切に説明しなければ、エージェントはそれらを正しく使いこなす方法を知ることができません。

> Without thinking through both, you won't be able to build a working agent.
> If you don't give the agent access to a correct set of tools, it will never be able to accomplish the objectives you give it.
> If you don't describe the tools well, the agent won't know how to use them properly.

LangChainは幅広い組み込みツールを提供しており、また、独自のツールやカスタムの説明を定義することも容易です。組み込みツールの完全なリストについては、[ツール統合セクション](/docs/integrations/tools/)をご覧ください。

> LangChain provides a wide set of built-in tools, but also makes it easy to define your own (including custom descriptions).
> For a full list of built-in tools, see the [tools integrations section](/docs/integrations/tools/)

### Toolkits | ツールキット

多くの一般的なタスクにおいて、エージェントには関連するツールのセットが必要です。このためにLangChainはツールキットの概念を提供しており、これは特定の目的を達成するために必要な約3-5個のツールのグループです。例えば、GitHubツールキットには、GitHubのイシューを検索するツール、ファイルを読むツール、コメントを投稿するツールなどが含まれています。

> For many common tasks, an agent will need a set of related tools.
> For this LangChain provides the concept of toolkits - groups of around 3-5 tools needed to accomplish specific objectives.
> For example, the GitHub toolkit has a tool for searching through GitHub issues, a tool for reading a file, a tool for commenting, etc.

LangChainは、始めるための幅広いツールキットを提供しています。組み込まれているツールキットの完全なリストについては、[ツールキット統合セクション](/docs/integrations/toolkits/)をご覧ください。

> LangChain provides a wide set of toolkits to get started.
> For a full list of built-in toolkits, see the [toolkits integrations section](/docs/integrations/toolkits/)

### AgentExecutor | AgentExecutor

エージェントエグゼキュータは、エージェントのランタイムです。これは実際にエージェントを呼び出し、選択したアクションを実行し、アクションの出力をエージェントに渡し、そして繰り返します。擬似コードでは、大まかに次のようになります：

> The agent executor is the runtime for an agent.
> This is what actually calls the agent, executes the actions it chooses, passes the action outputs back to the agent, and repeats.
> In pseudocode, this looks roughly like:

```python
next_action = agent.get_action(...)
while next_action != AgentFinish:
    observation = run(next_action)
    next_action = agent.get_action(..., next_action, observation)
return next_action
```

これは単純に見えるかもしれませんが、このランタイムがあなたに代わって処理してくれるいくつかの複雑な部分があります。これには以下のものが含まれます：

> While this may seem simple, there are several complexities this runtime handles for you, including:

1. エージェントが存在しないツールを選択した場合の対処方法

   > Handling cases where the agent selects a non-existent tool

2. ツールがエラーを出した場合の対処方法

   > Handling cases where the tool errors

3. エージェントがツールの呼び出しに解析できない出力を生成した場合の対処方法

   > Handling cases where the agent produces output that cannot be parsed into a tool invocation

4. すべてのレベル（エージェントの決定、ツールの呼び出し）におけるログ記録と可観測性は、標準出力および/または[LangSmith](/docs/langsmith)へ出力されます。

   > Logging and observability at all levels (agent decisions, tool calls) to stdout and/or to [LangSmith](/docs/langsmith).


### Other types of agent runtimes | その他の種類のエージェントランタイム

`AgentExecutor` クラスは、LangChainによってサポートされる主要なエージェントランタイムです。しかし、私たちは他にも、より実験的なランタイムをサポートしています。これには以下のものが含まれます：

> The `AgentExecutor` class is the main agent runtime supported by LangChain.
> However, there are other, more experimental runtimes we also support.
> These include:

* [Plan-and-execute Agent](/docs/use_cases/more/agents/autonomous_agents/plan_and_execute)（計画と実行エージェント）

  > [Plan-and-execute Agent](/docs/use_cases/more/agents/autonomous_agents/plan_and_execute)

* [ベビーAGI](/docs/use_cases/more/agents/autonomous_agents/baby_agi)

  > [Baby AGI](/docs/use_cases/more/agents/autonomous_agents/baby_agi)

* [Auto GPT](/docs/use_cases/more/agents/autonomous_agents/autogpt)

  > [Auto GPT](/docs/use_cases/more/agents/autonomous_agents/autogpt)


また、以下で説明する方法に従って、独自のカスタム実行ロジックを作成することもできます。

> You can also always create your own custom execution logic, which we show how to do below.

## Get started | 始めましょう

LangChain Expression Language (LCEL) を使ってエージェントフレームワークを深く理解するために、一からエージェントを構築しましょう。エージェント自体を構築し、カスタムツールを定義し、カスタムループでエージェントとツールを実行する必要があります。最後に、実行をより簡単にするための標準LangChain `AgentExecutor` の使用方法を紹介します。

> To best understand the agent framework, lets build an agent from scratch using LangChain Expression Language (LCEL).
> We'll need to build the agent itself, define custom tools, and run the agent and tools in a custom loop. At the end we'll show how to use the standard LangChain `AgentExecutor` to make execution easier.

知っておくべきいくつかの重要な用語（およびスキーマ）：

> Some important terminology (and schema) to know:

1. `AgentAction`: これはエージェントが取るべきアクションを表すデータクラスです。`tool` プロパティ（呼び出すべきツールの名前）と `tool_input` プロパティ（そのツールへの入力）を持っています

   > `AgentAction`: This is a dataclass that represents the action an agent should take. It has a `tool` property (which is the name of the tool that should be invoked) and a `tool_input` property (the input to that tool)

2. `AgentFinish`: これはエージェントが処理を完了し、ユーザーに結果を返すべきことを示すデータクラスです。`return_values`というパラメータを持っており、これは返すべきデータを含む辞書です。しばしばこの辞書は`output`という一つのキーのみを持ち、そのキーは文字列です。そのため、多くの場合、この`output`キーのみが返されます。

   > `AgentFinish`: This is a dataclass that signifies that the agent has finished and should return to the user. It has a `return_values` parameter, which is a dictionary to return. It often only has one key - `output` - that is a string, and so often it is just this key that is returned.

3. `intermediate_steps`: これらは、過去のエージェントのアクションとそれに対応する出力を表しており、受け渡されていきます。エージェントがこれまでに行った作業を把握するために、これらを将来のイテレーションに渡すことが重要です。これは`List[Tuple[AgentAction, Any]]`として型付けされています。観測は最大限の柔軟性を持たせるために現在`Any`型として残されていますが、実際には、これは通常文字列です。

   > `intermediate_steps`: These represent previous agent actions and corresponding outputs that are passed around. These are important to pass to future iteration so the agent knows what work it has already done. This is typed as a `List[Tuple[AgentAction, Any]]`. Note that observation is currently left as type `Any` to be maximally flexible. In practice, this is often a string.


### Setup: LangSmith | セットアップ：LangSmith

定義によれば、エージェントはユーザーに対する出力を返す前に、自己決定的かつ入力依存の一連のステップを踏みます。これにより、これらのシステムのデバッグは特に難しくなり、可観測性が特に重要になります。[LangSmith](/docs/langsmith)は、このような場合に特に有用です。

> By definition, agents take a self-determined, input-dependent sequence of steps before returning a user-facing output. This makes debugging these systems particularly tricky, and observability particularly important. [LangSmith](/docs/langsmith) is especially useful for such cases.

LangChainを使用して構築する際、組み込みエージェントやLCELで構築されたカスタムエージェントは、LangSmithで自動的にトレースされます。また、`AgentExecutor`を使用すれば、エージェントの計画ステップだけでなく、ツールの入力と出力も完全にトレースされます。

> When building with LangChain, any built-in agent or custom agent built with LCEL will automatically be traced in LangSmith. And if we use the `AgentExecutor`, we'll get full tracing of not only the agent planning steps but also the tool inputs and outputs.

LangSmithをセットアップするためには、以下の環境変数を設定する必要があります：

> To set up LangSmith we just need set the following environment variables:

```bash
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="<your-api-key>"
```

### Define the agent | エージェントを定義する

まず、私たちのエージェントを作成する必要があります。これは次に取る行動を決定するための連鎖的なプロセスを担当します。

> We first need to create our agent.
> This is the chain responsible for determining what action to take next.

この例では、OpenAI Function Callingを使用してこのエージェントを作成します。
**これは一般的にエージェントを作成する最も信頼性の高い方法です。**

> In this example, we will use OpenAI Function Calling to create this agent.
> **This is generally the most reliable way to create agents.**

このガイドでは、カスタムツールにアクセスできるカスタムエージェントを構築します。実際の多くの使用例では、エージェントまたはツールのいずれかをカスタマイズする必要があります。この例を選んだのは、そのためです。単語の長さを計算するシンプルなツールを作成しましょう。これは、トークン化のためにLLMが間違えることがあるので、実際に役立ちます。最初は記憶機能なしで作成しますが、その後、記憶機能をどのように追加するかを示します。記憶機能は対話を可能にするために必要です。

> For this guide, we will construct a custom agent that has access to a custom tool.
> We are choosing this example because for most real world use cases you will NEED to customize either the agent or the tools.
> We'll create a simple tool that computes the length of a word.
> This is useful because it's actually something LLMs can mess up due to tokenization.
> We will first create it WITHOUT memory, but we will then show how to add memory in.
> Memory is needed to enable conversation.

まず、エージェントを制御するために使用する言語モデルをロードしましょう。

> First, let's load the language model we're going to use to control the agent.




In [1]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

文字列「educa」の文字を数えるのに苦労していることがわかります。

> We can see that it struggles to count the letters in the string "educa".




In [12]:
llm.invoke("how many letters in the word educa?")

AIMessage(content='There are 6 letters in the word "educa".')

次に、使用するいくつかのツールを定義しましょう。
渡された単語の長さを計算する非常にシンプルなPython関数を書いてみましょう。

> Next, let's define some tools to use.
> Let's write a really simple Python function to calculate the length of a word that is passed in.




In [2]:
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


tools = [get_word_length]

それでは、プロンプトを作成しましょう。
OpenAI Function Callingはツール使用に特化してファインチューニングされているので、推論の方法や出力フォーマットについての指示はほとんど必要ありません。
私たちは二つの入力変数、`input`と`agent_scratchpad`を用意するだけです。`input`はユーザーの目的を含む文字列であるべきです。`agent_scratchpad`は、以前のエージェントツールの呼び出しとそれに対応するツールの出力を含むメッセージのシーケンスであるべきです。

> Now let us create the prompt.
> Because OpenAI Function Calling is finetuned for tool usage, we hardly need any instructions on how to reason, or how to output format.
> We will just have two input variables: `input` and `agent_scratchpad`. `input` should be a string containing the user objective. `agent_scratchpad` should be a sequence of messages that contains the previous agent tool invocations and the corresponding tool outputs.




In [3]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

エージェントはどのようにして使用可能なツールを知るのでしょうか？この場合、私たちは関数を別の引数として受け取り、それらの関数をいつ呼び出すべきかを特に訓練されているOpenAIの関数呼び出しLLMsに頼っています。

> How does the agent know what tools it can use?
> In this case we're relying on OpenAI function calling LLMs, which take functions as a separate argument and have been specifically trained to know when to invoke those functions.

エージェントにツールを渡すためには、それらをOpenAI関数の形式にフォーマットし、モデルに渡すだけです。（関数を`bind`することにより、モデルが呼び出されるたびにそれらが渡されるようにしています。）

> To pass in our tools to the agent, we just need to format them to the OpenAI function format and pass them to our model. (By `bind`-ing the functions, we're making sure that they're passed in each time the model is invoked.)




In [4]:
from langchain.tools.render import format_tool_to_openai_function

llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

これらの要素を組み合わせて、私たちはこれでエージェントを作成することができます。最後に2つのユーティリティ関数をインポートしましょう：1つは中間ステップ（エージェントのアクションとツールの出力のペア）をモデルに送信できる入力メッセージにフォーマットするコンポーネント、もう1つは出力メッセージをエージェントのアクションまたはエージェントの終了に変換するコンポーネントです。

> Putting those pieces together, we can now create the agent.
> We will import two last utility functions: a component for formatting intermediate steps (agent action, tool output pairs) to input messages that can be sent to the model, and a component for converting the output message into an agent action/agent finish.




In [5]:
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

エージェントを準備できたので、早速試してみましょう！簡単な質問と空の中間ステップを渡して、どのような結果が返ってくるか見てみましょう：

> Now that we have our agent, let's play around with it!
> Let's pass in a simple question and empty intermediate steps and see what it returns:




In [7]:
agent.invoke({"input": "how many letters in the word educa?", "intermediate_steps": []})

AgentActionMessageLog(tool='get_word_length', tool_input={'word': 'educa'}, log="\nInvoking: `get_word_length` with `{'word': 'educa'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "word": "educa"\n}', 'name': 'get_word_length'}})])

それが `AgentAction` を取るために応答していることがわかります（実際には、完全なメッセージログを追跡する `AgentAction` のサブクラスである `AgentActionMessageLog` です）。

> We can see that it responds with an `AgentAction` to take (it's actually an `AgentActionMessageLog` - a subclass of `AgentAction` which also tracks the full message log).

LangSmithを設定していれば、シーケンス内の各ステップの入力と出力を検査することができるトレースが表示されます: https://smith.langchain.com/public/04110122-01a8-413c-8cd0-b4df6eefa4b7/r

> If we've set up LangSmith, we'll see a trace that let's us inspect the input and output to each step in the sequence: https://smith.langchain.com/public/04110122-01a8-413c-8cd0-b4df6eefa4b7/r

### Define the runtime | ランタイムを定義する

これは最初のステップに過ぎません - これに対応するランタイムを書く必要があります。最もシンプルなランタイムは、エージェントを呼び出し、アクションを実行し、`AgentFinish`が返されるまでそのプロセスを繰り返すものです。以下でそのコードを書いてみましょう：

> So this is just the first step - now we need to write a runtime for this.
> The simplest one is just one that continuously loops, calling the agent, then taking the action, and repeating until an `AgentFinish` is returned.
> Let's code that up below:




In [9]:
from langchain_core.agents import AgentFinish

user_input = "how many letters in the word educa?"
intermediate_steps = []
while True:
    output = agent.invoke(
        {
            "input": user_input,
            "intermediate_steps": intermediate_steps,
        }
    )
    if isinstance(output, AgentFinish):
        final_result = output.return_values["output"]
        break
    else:
        print(f"TOOL NAME: {output.tool}")
        print(f"TOOL INPUT: {output.tool_input}")
        tool = {"get_word_length": get_word_length}[output.tool]
        observation = tool.run(output.tool_input)
        intermediate_steps.append((output, observation))
print(final_result)

TOOL NAME: get_word_length
TOOL INPUT: {'word': 'educa'}
There are 5 letters in the word "educa".


わーい！動いてる。

> Woo! It's working.

### Using AgentExecutor | AgentExecutorの使用

これを少し簡単にするために、`AgentExecutor` クラスをインポートして使用できます。これにより、上記のすべてが一つにまとめられ、エラーハンドリング、早期停止、トレーシング、その他の利便性を向上させる改善が加えられ、書く必要のあるセーフガードが減少します。

> To simplify this a bit, we can import and use the `AgentExecutor` class.
> This bundles up all of the above and adds in error handling, early stopping, tracing, and other quality-of-life improvements that reduce safeguards you need to write.




In [10]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

さあ、試してみましょう！

> Now let's test it out!




In [11]:
agent_executor.invoke({"input": "how many letters in the word educa?"})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


{'input': 'how many letters in the word educa?',
 'output': 'There are 5 letters in the word "educa".'}

トレースを見ると、エージェントの呼び出しやツールの起動が自動的に記録されていることがわかります：https://smith.langchain.com/public/957b7e26-bef8-4b5b-9ca3-4b4f1c96d501/r

> And looking at the trace, we can see that all of our agent calls and tool invocations are automatically logged: https://smith.langchain.com/public/957b7e26-bef8-4b5b-9ca3-4b4f1c96d501/r




### Adding memory | メモリの追加

素晴らしいことです - 私たちにはエージェントがいます！ しかし、このエージェントはステートレスで、以前のインタラクションについて何も記憶していません。これでは、フォローアップの質問を簡単にはできません。記憶機能を追加して、この問題を解決しましょう。

> This is great - we have an agent!
> However, this agent is stateless - it doesn't remember anything about previous interactions.
> This means you can't ask follow up questions easily.
> Let's fix that by adding in memory.

これを行うためには、二つのことを行う必要があります：

> In order to do this, we need to do two things:

1. プロンプトにメモリ変数を格納するための場所を追加してください

   > Add a place for memory variables to go in the prompt

2. チャット履歴を記録しておく

   > Keep track of the chat history


まず、プロンプトにメモリースペースを追加しましょう。
これを行うには、`"chat_history"` というキーを持つメッセージのためのプレースホルダーを追加します。
新しいユーザー入力の上にこれを配置することに注意してください（会話の流れに沿って配置するためです）。

> First, let's add a place for memory in the prompt.
> We do this by adding a placeholder for messages with the key `"chat_history"`.
> Notice that we put this ABOVE the new user input (to follow the conversation flow).




In [13]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

その後、チャット履歴を追跡するためのリストを設定できます

> We can then set up a list to track the chat history




In [14]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

それでは、全てをまとめましょう！

> We can then put it all together!




In [15]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

実行中には、チャット履歴として入力と出力を追跡する必要があります

> When running, we now need to track the inputs and outputs as chat history




In [17]:
input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


> Entering new AgentExecutor chain...
No, "educa" is not a real word in English.

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='There are 5 letters in the word "educa".')],
 'output': 'No, "educa" is not a real word in English.'}

LangSmithのトレースはこちらです：https://smith.langchain.com/public/1e1b7e07-3220-4a6c-8a1e-f04182a755b3/r

> Here's the LangSmith trace: https://smith.langchain.com/public/1e1b7e07-3220-4a6c-8a1e-f04182a755b3/r




## Next Steps | 次のステップ

素晴らしい！あなたは最初のエンドツーエンドエージェントを実行しました。
さらに詳しく学びたい場合は、以下のことができます：

> Awesome! You've now run your first end-to-end agent.
> To dive deeper, you can:

* サポートされているさまざまな[エージェントの種類](/docs/modules/agents/agent_types/)をご覧ください

  > Check out all the different [agent types](/docs/modules/agents/agent_types/) supported

* [AgentExecutor](/docs/modules/agents/how_to/)のすべてのコントロールを学びましょう。

  > Learn all the controls for [AgentExecutor](/docs/modules/agents/how_to/)

* [ツール](/docs/modules/agents/tools/)の使い方と、すべての[ツールの統合](/docs/integrations/tools)を探求しましょう。

  > Explore the how-to's of [tools](/docs/modules/agents/tools/) and all the [tool integrations](/docs/integrations/tools)

* 私たちが提供する既製の[ツールキット](/docs/integrations/toolkits/)の完全なリストをご覧ください

  > See a full list of all the off-the-shelf [toolkits](/docs/integrations/toolkits/) we provide



